In [6]:
class Point:
    def __init__(self,x: int, y: int):
        self.x = x
        self.y = y

def cell(x, y):
    if (y < 0 or y >= H or x < 0 or x >= W):
        return -1
    return board[y][x]

def boundary_path(x, y, c):
    direction = "s"
    x0 = x
    y0 = y
    y += 1
    path = []
    while True:
        checked = False
        
        if (cell(x, y) == c):
            if (cell(x, y-1) not in [-1,c]):
                checked=True
                res[y-1][x] = 1

            if (cell(x, y+1) not in [-1,c]):
                checked=True
                res[y+1][x] = 1

            if (cell(x-1, y) not in [-1,c]):
                checked=True
                res[y][x-1] = 1

            if (cell(x+1, y) not in [-1,c]):
                checked=True
                res[y][x+1] = 1

            if (checked is True):
                res[y][x] = 1

        print((x,y,direction))

        if (x==x0 and y==y0):
            break
        
        match direction:
            case "n":
                l = 1 if (cell(x+1, y-1) == c) else 0
                r = 2 if (cell(x, y-1) == c) else 0
                direction = ["w", "n", "e", "e"][l + r]
            case "e":
                l = 1 if (cell(x, y-1) == c) else 0
                r = 2 if (cell(x, y) == c) else 0
                direction = ["n", "e", "s", "s"][l + r]
            case "s":
                l = 1 if (cell(x, y) == c) else 0
                r = 2 if (cell(x-1, y) == c) else 0
                direction = ["e", "s", "w", "w"][l + r]
            case "w":
                l = 1 if (cell(x-1, y) == c) else 0
                r = 2 if (cell(x-1, y-1) == c) else 0
                direction = ["s", "w", "n", "n"][l + r]
        
        match direction:
            case "n":
                if y > 0:
                    y -= 1
                else:
                    x -= 1
                    direction = "w"
            case "e":
                if x < W-1:
                    x += 1
                else: 
                    y -= 1
                    direction = "n"
            case "s":
                if y < H-1:
                    y += 1
                else:
                    x += 1
                    direction = "e"
            case "w":
                if x > 0:
                    x -= 1
                else:
                    y += 1
                    direction = "s"

In [7]:
res = [[0]*W for _ in range(H)]
def boundary_finding():
    done = []
    for x in range(W):
        for y in range(H):
            c = board[y][x]
            if (c not in done and c == 0):
                done.append(c)
                boundary_path(x,y,c)

boundary_finding()

(0, 1, 's')
(0, 2, 's')
(0, 3, 's')
(1, 3, 'e')
(2, 3, 'e')
(2, 2, 'n')
(1, 2, 'w')
(1, 1, 'n')
(2, 1, 'e')
(2, 0, 'n')
(1, 0, 'w')
(0, 0, 'w')


In [35]:
global W, H
W = 6
H = 4
global board, res

board = [
    [0, 0, 1, 1, 2, 2],
    [0, 0, 1, 2, 2, 2],
    [0, 0, 2, 2, 3, 2],
    [0, 0, 0, 0, 0, 0]
]

In [2]:
def cell(x, y):
    if (y < 0 or y >= H or x < 0 or x >= W):
        return -1
    return board[y][x]

def boundary_finding(region):
    done = []
    for x in range(W):
        for y in range(H):
            m = cell(x,y)
            t = cell(x,y-1)
            r = cell(x+1,y)
            l = cell(x-1, y)
            b = cell(x, y+1)
            tmp = {-1, t, r, l, b, m}
            if len(tmp) <= 2:
                continue
            if (m in region):
                res[y][x] = 1

In [2]:
def print_sol(sol):
    for row in sol:
        print(row)

In [20]:
import numpy as np

x = np.zeros((3, 3), dtype=int)
x[1,] = 1

x = np.roll(x, 2, axis = 0)

x

array([[1, 1, 1],
       [0, 0, 0],
       [0, 0, 0]])

In [8]:
res = [[0]*W for _ in range(H)]
boundary_finding([0])

print_sol(board)
print()
print_sol(res)

[0, 0, 1, 1, 2, 2]
[0, 0, 1, 2, 2, 2]
[0, 0, 2, 2, 3, 0]
[0, 0, 0, 0, 0, 0]

[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 1]
[0, 0, 1, 1, 1, 0]


In [1]:
from scipy.ndimage.morphology import binary_dilation
import numpy as np

/var/folders/jz/wtln99xj6vv6wwcgzd39rq7h0000gn/T/ipykernel_5237/616775809.py:1: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_dilation


In [11]:
def get_boundary(region, region1, region2):
    bound1 = np.isin(region, region1)
    bound2 = np.isin(region, region2)

    k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1 # for 8-connected
    bound1 = binary_dilation(bound1==0, k) & bound1
    bound2 = binary_dilation(bound2==0, k) & bound2

    top = np.roll(bound2, -1, axis=0)
    top[-1,] = 0

    bottom = np.roll(bound2, 1, axis=0)
    bottom[0,] = 0
    
    left = np.roll(bound2, -1, axis=1)
    left[:,-1] = 0

    right = np.roll(bound2, 1, axis=1)
    right[:,0] = 0

    bound1 = bound1 * top + bound1 * bottom + bound1 * left + bound1 * right
    bound2 = np.roll(bound1 * top, 1, axis=0) + np.roll(bound1 * bottom, -1, axis=0) + np.roll(bound1 * left, 1, axis=1) + np.roll(bound1 * right, -1, axis=1)
    res = bound1 + bound2

    return res
        

In [33]:
def get_boundary_1(region):
    no_tiles = np.random.randint(2, 3)

    k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1 # for 8-connected
    bound = np.isin(region, 0)
    tmp = ~bound
    
    res = np.zeros((H, W), dtype=bool)
    for _ in range(no_tiles):
        top = np.roll(bound, -1, axis=0)
        top[-1,] = 0

        bottom = np.roll(bound, 1, axis=0)
        bottom[0,] = 0
        
        left = np.roll(bound, -1, axis=1)
        left[:,-1] = 0

        right = np.roll(bound, 1, axis=1)
        right[:,0] = 0

        bound = top + bottom + left + right
        res += bound
    
    return res * tmp

In [36]:
print_sol(board)

get_boundary_1(board)

[0, 0, 1, 1, 2, 2]
[0, 0, 1, 2, 2, 2]
[0, 0, 2, 2, 3, 2]
[0, 0, 0, 0, 0, 0]


array([[False, False,  True,  True, False, False],
       [False, False,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True],
       [False, False, False, False, False, False]])